In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
if "KERAS_BACKEND" not in os.environ:
    # set this to "torch", "tensorflow", or "jax"
    os.environ["KERAS_BACKEND"] = "tensorflow"

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from functools import partial

np.set_printoptions(suppress=True)

In [4]:
from boundary_conditions import bound_agent_position
from priors import complete_pooling_prior

In [5]:
import keras

import bayesflow as bf
import tensorflow as tf

2025-07-27 10:38:41.498602: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-27 10:38:41.577490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753630721.608925   17436 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753630721.617852   17436 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753630721.681925   17436 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [6]:
bf.__version__

'2.0.6'

In [7]:
# Trying numba again with a wrapper
from numba import njit

## Generative Model Definition

The movement of any agent $a = 1, ..., A$ is both related to: 1) its interaction with surrounding neighbors $i = 1, ..., I$, which we call *internal influence*, and 2) their motivation to the surrounding spatial objects $b = 1, ..., B$, which we call *external influence*. These influences are modulated by a stationary weight, $w_a$:

\begin{equation}
    \theta_{a, t} = w_a \theta_{a|j, t} + (1 - w_a) \theta_{a|i, t}.
\end{equation}

### Meta-Variables

First, we define some meta-variables, such as the number of agents to simulate, the number of spatial beacons present in the environment, etc.

In [8]:
num_agents = 12
num_beacons = 2
room_size = (8., 10.)
world_size = 25.

### Agent Initialization

In [9]:
@njit
def initialize_agents(
    num_agents: int = 12,
    room_size: tuple = (8., 10.),
):
    """
    Generate random positions and orientations for agents.

    Parameters
    ----------
    num_agents : int, optional
        Number of agents to generate (default is 100).
    room_size : float, optional
        The size of the boundary within which positions are generated (default is 100.0).

    Returns
    -------
    tuple of np.ndarray
        A tuple containing the positions (np.ndarray) and orientations (np.ndarray) of the agents.
    """

    # Generate random positions within the boundary size centered at 0
    x = (np.random.random(size=num_agents).astype(np.float32) - 0.5) * room_size[0]
    y = (np.random.random(size=num_agents).astype(np.float32) - 0.5) * room_size[1]
    positions = np.vstack((x, y)).T

    # Generate random orientations (angles in radians between 0 and 2*pi)
    rotations = np.random.random(size=(num_agents, )).astype(np.float32) * np.pi * 2

    return positions.astype(np.float32), rotations.astype(np.float32)

In [10]:
p, r = initialize_agents(room_size=room_size, num_agents=12)
p

array([[-0.15715909,  1.0016245 ],
       [ 2.832931  ,  1.8684179 ],
       [-0.39079976,  4.3459663 ],
       [ 2.8210168 ,  2.6284523 ],
       [ 3.7776475 , -3.410761  ],
       [ 2.0081534 ,  4.6942735 ],
       [ 2.1511912 ,  4.301675  ],
       [-0.7819321 ,  0.8893734 ],
       [ 0.97294426,  3.0981631 ],
       [-0.01738811,  1.0164589 ],
       [ 1.0862064 ,  1.1993265 ],
       [ 1.6297746 ,  3.2795982 ]], dtype=float32)

### Beacon Initialization

In [11]:
@njit
def initialize_beacons(
        num_beacons = 10,
        room_sensing_range = 50.
):

    """
    Initialize beacons following a uniform distribution scaled to the room's sensing boundary

    Parameters
    ----------
    num_beacons : int, default: 10
        Number of beacons to initialize.
    room_sensing_range : float, default: 50.0
        Sensing distance of the room for the beacons to matter.

    Returns
    -------
    beacons      : np.ndarray of shape (num_beacons, 2)
        Initial positions of the beacons.
    """

    beacons = (np.random.random(size=(num_beacons, 2)) - 0.5) * room_sensing_range
    return beacons.astype(np.float32)

In [12]:
beacon_positions = initialize_beacons(num_beacons=1, room_sensing_range=world_size)
beacon_positions

array([[-10.086244,  -7.680403]], dtype=float32)

## External Influence: drift-diffusion vector

We want to compute the influence of agent movement direction within a single time step. For this, we specify our internal influence as a 2D drift diffusion model, where the agents are approach a spatial beacon within the room's boundary by reorienting its locomotive direction.

\begin{equation}
    \theta_{a|j, t} = \theta_{a|j, t-1} + \omega_a \mathrm{d}t + \mathrm{d}\phi_t,
\end{equation}

\begin{align}
    \mathrm{d}\mathbf{x}_{a|j, t}
    &= v_{a|j}\mathrm{d}t \frac{\mathbf{x}_{a|j}}{||\mathbf{x}_{a|j}||} + \sigma_{a|j} \mathrm{d}\mathrm{W}_t \\
    &= v_{a}\mathrm{d}t
    \begin{bmatrix}
        \cos \theta_{a|j, t} \\
        \sin \theta_{a|j, t}
    \end{bmatrix} + \sigma_{a|j} \mathrm{d}\mathrm{W}_t,% \sqrt{\mathrm{d}t} Z_t.
\end{align}

In [13]:
@njit
def external_influence(
    agent_position,
    beacon_position,
    noise = False,
    noise_amplitude = 0.01
):
    """
    Generate a drift-diffusion vector in 2D space for a single agent
    based on a target location (in this case, the position of a beacon).

    Parameters
    ----------
    agent_position : np.ndarray
        The position of the agent.
    target_position : np.ndarray
        The position of the target beacon.
    focus : float, optional
        The dispersion of a von Mises distribution for rotational noise influenced by the neighbors.
        The higher the value is, the less perturbation there would be.
    noise: bool, optional
        Whether the focus is interpreted as noise amplitude.

    Returns
    -------
    np.ndarray
        A 2D vector representing the drift-diffusion process towards the target (beacon).
    """
    # Calculate the angle towards the beacon (in radian)
    beacon_direction = np.arctan2(
        beacon_position[1] - agent_position[1],
        beacon_position[0] - agent_position[0]
    )

    # Generate a random direction with drift around the target angle
    if noise:
        beacon_direction = beacon_direction + (np.random.random() - 0.5) * noise_amplitude
        # beacon_direction = beacon_direction + np.random.vonmises(0., 8.) * noise_amplitude

    # Convert the angle to a unit vector in 2D space
    v = np.array([np.cos(beacon_direction), np.sin(beacon_direction)], dtype=np.float32)

    return v

## Internal Influence: particle dynamics

Its influence by a collective group of agents is modeled as a self-propelling particle system, as expressed in the Vicsek model:

\begin{align}
    \theta_{a|i, t} &= \langle \theta_{i, t}\rangle_{|\mathbf{x}_a - \mathbf{x}_i| < r_a, i \in I} + \eta_{a,t-1}, \\
    \mathrm{d} \mathbf{x}_{a|i,t} &= v_{a,t} \mathrm{d}t
    \begin{bmatrix}
        \cos \theta_{a|i, t} \\
        \sin \theta_{a|i, t}
    \end{bmatrix},
\end{align}

In [14]:
@njit
def internal_influence(
        self_position,
        other_positions,
        other_rotations,
        sensing_radius = 1.5,
        focus = 0.01,
        noise = False
):
    """
    Generate an influence vector for a single agent
    based on the angular component of the Vicsek model.

    Parameters
    ----------
    self_position : np.ndarray of shape (2,)
        A 2D vector representing the position of the agent
    other_positions : np.ndarray of shape (2,)
        A 2D vector representing the positions of the neighboring agents.
    other_rotations : np.ndarray of shape (2,)
        A 2D vector representing the rotations of the neighboring agents.
    sensing_radius : float
        The sensing radius within which agents interact with their neighbors.
    focus : float, optional
        The dispersion of a von Mises distribution for rotational noise influenced by the neighbors.
        The higher the value is, the less perturbation there would be.
    noise: bool, optional
        Whether the focus is interpreted as noise amplitude.

    Returns
    -------
    np.ndarray
        A 2D unit vector representing the averaged influence direction with added noise.
    """

    neighbor_rotations = []

    for i in range(len(other_positions)):
        dx = other_positions[i, 0] - self_position[0]
        dy = other_positions[i, 1] - self_position[1]
        d = (dx ** 2 + dy ** 2) ** 0.5

        if d <= sensing_radius and d > 0:
            neighbor_rotations.append(other_rotations[i])

    if len(neighbor_rotations) == 0:
        return np.array([0.0, 0.0], dtype=np.float32)

    neighbor_rotations = np.array(neighbor_rotations)
    averaged_rotation = np.sum(neighbor_rotations) / len(neighbor_rotations)

    if noise:
        deviation = (np.random.random() - 0.5) * focus
    else:
        deviation = np.random.vonmises(mu=0., kappa=4.) * focus
    direction = averaged_rotation + deviation

    v = np.array([np.cos(direction), np.sin(direction)], dtype=np.float32)

    return v

## Putting everything together: combined influences

The combined influences allow us to update the agents' positions and rotations together.

In [15]:
@njit
def count_neighbors(self_position, other_positions, sensing_radius = 1.5):
    """
    Helper function that counts the number of neighbors

    Parameters
    ----------
    self_position   : np.ndarray of size (2)
        The position of the agent itself
    other_positions : np.ndarray of size (num_agents, 2)
        The positions of all agents
    sensing_radius  : float, default: 1.5
        The sensing radius of the agent

    Returns
    -------
    num_neighbors   : int, default: 0
        The number of neighbors within the agent's sensing radius.
    """

    num_neighbors = 0

    for i in range(len(other_positions)):
        dx = other_positions[i, 0] - self_position[0]
        dy = other_positions[i, 1] - self_position[1]
        d = (dx ** 2 + dy ** 2) ** 0.5

        if d <= sensing_radius and d > 0:
            num_neighbors += 1

    return num_neighbors

In [16]:
num_neighbors = count_neighbors(p[8], p)
num_neighbors

1

In [17]:
@njit
def combined_influences(
    agent_positions: np.ndarray = None,
    agent_rotations: np.ndarray = None,
    beacon_positions: np.ndarray = None,
    velocity: float = 1.0,
    sensing_radius: float = 2.5,
    dt: float = 0.1,
    influence_weight: float = 0.5,
    internal_focus: float = 0.1
):
    """
    Update the positions and orientations of a single agent
    based on velocity and influence vectors.

    Parameters
    ----------
    agent_positions : np.ndarray
        Current positions of the agents.
    agent_rotations : np.ndarray
        Current orientations of the agents.
    beacon_positions : np.ndarray
        Positions of the beacons.
    velocity : float, optional
        The speed at which agents move (default is 1.0).
    sensing_radius : float, optional
        The sensing radius within which agents interact with their neighbors.
    dt : float, optional
        The time step for updating positions and orientations (default is 0.1).
    influence_weight : float, optional
        The weight of influence_vector1 in determining new orientations (default is 0.7).
    external_focus : float, optional
        Concentration of the agent's rotational noise influenced by the beacons
    internal_focus : float, optional
        Concentration of the agent's rotational noise influenced by the neighbors

    Returns
    -------
    tuple of np.ndarray
        Updated positions (np.ndarray) and orientations (np.ndarray) of the agents.
    """

    assert (len(agent_positions) == len(agent_rotations))

    num_agents = agent_positions.shape[0]
    num_beacons = beacon_positions.shape[0]

    # Create new numpy arrays for the updated agent positions and rotations
    new_agent_positions = np.zeros((num_agents, 2))
    new_agent_rotations = np.zeros((num_agents, ))
    num_neighbors = np.zeros((num_agents, ))


    for i in range(num_agents):

        num_neighbors[i] = count_neighbors(agent_positions[i], agent_positions)

        # Generate the ddm vector for the agent based on its closest beacon
        distance_to_beacon = []

        for b in range(num_beacons):
            bx = beacon_positions[b, 0] - agent_positions[i, 0]
            by = beacon_positions[b, 1] - agent_positions[i, 1]
            distance_to_beacon.append((bx * bx + by * by) ** 0.5)

        beacon_id = np.argmin(np.array(distance_to_beacon))

        ddm_vector = external_influence(
            agent_positions[i],
            beacon_positions[beacon_id],
            #focus=external_focus
        )

        # Generate the vicsek vector for the agent based on its neighbors (all agents)
        vicsek_vector = internal_influence(
            self_position=agent_positions[i],
            other_positions=agent_positions,
            other_rotations=agent_rotations,
            sensing_radius=sensing_radius,
            focus=internal_focus
        )

        # Update orientations based on two influence vectors
        ddm_influence = np.arctan2(ddm_vector[1], ddm_vector[0])
        vicsek_influence = np.arctan2(vicsek_vector[1], vicsek_vector[0])

        # Combine influences to update orientations with different weights
        new_agent_rotations[i] = agent_rotations[i] + (influence_weight * ddm_influence + (1 - influence_weight) * vicsek_influence) * dt

        # Ensure orientations are within the range [0, 2*pi]
        new_agent_rotations[i] = np.mod(new_agent_rotations[i], 2 * np.pi)

        # Update positions based on current orientations
        new_agent_positions[i, 0] = agent_positions[i, 0] + velocity * np.cos(new_agent_rotations[i].item()) * dt
        new_agent_positions[i, 1] = agent_positions[i, 1] + velocity * np.sin(new_agent_rotations[i].item()) * dt

        new_agent_positions[i] = bound_agent_position(new_agent_positions[i], room_size=room_size)

    return new_agent_positions, new_agent_rotations, num_neighbors

In [18]:
agent_positions, agent_rotations = initialize_agents(12, room_size=room_size)
beacon_positions = initialize_beacons(num_beacons=2)
new_agent_positions, new_agent_rotations, num_neighbors = combined_influences(agent_positions, agent_rotations, beacon_positions)

In [19]:
np.concatenate([agent_positions, new_agent_positions], axis=1)

array([[ 2.04276466,  3.89841747,  2.13973271,  3.87397984],
       [-0.96194196,  3.52170658, -0.98370297,  3.424103  ],
       [-2.1507237 ,  2.72945166, -2.23885858,  2.68220504],
       [ 1.72830009,  0.89522064,  1.80733946,  0.95648056],
       [ 1.47903061, -3.74447107,  1.55036913, -3.67439387],
       [-2.25255322,  1.87738419, -2.35248808,  1.88099321],
       [ 0.43127775,  4.34173489,  0.5240099 ,  4.30430832],
       [ 0.60735226, -4.6319375 ,  0.65531996, -4.71968201],
       [ 0.70764017,  4.65200663,  0.61605027,  4.6921475 ],
       [-1.44591546,  3.64905643, -1.36200107,  3.70344738],
       [ 1.28609562, -4.53881311,  1.34047441, -4.62273538],
       [-3.26457262, -3.77155113, -3.27925952, -3.67263554]])

In [20]:
np.vstack([agent_rotations, new_agent_rotations]).T

array([[6.0382185 , 6.03630881],
       [4.53555012, 4.49302377],
       [3.41017032, 3.63367953],
       [0.56766957, 0.65934497],
       [0.72159046, 0.77647918],
       [3.1696465 , 3.10549459],
       [5.94487906, 5.89958053],
       [4.97673845, 5.21267561],
       [2.70993209, 2.72853827],
       [0.57989484, 0.57508896],
       [5.05512714, 5.28733312],
       [1.64087987, 1.71819854]])

In [21]:
num_neighbors

array([0., 2., 3., 0., 2., 1., 1., 2., 1., 2., 2., 0.])

## Simulation Loop

The update allows us to continuously simulate the agents' positions and rotations at a given interval

In [22]:
@njit(parallel=True)
def simulator_fun(
    batch_size: int = 1,
    theta = None,
    num_agents: int = 12,
    num_beacons: int = 1,
    room_size: tuple = (8, 10),
    velocity: float = 1.0,
    dt: float = 0.001,
    influence_weight: float = 0.7,
    sensing_radius: float = 10.0,
    internal_focus: float = 0.1,
    time_horizon: float = 30.
):
    """
    Run the simulation and store the time series of positions and orientations of agents.

    Parameters
    ----------
    theta : np.ndarray
        Prior parameters specifying the internal properties of the agents
    num_agents : int, optional
        Number of agents to generate (default is 100).
    num_beacons : int, optional
        Number of beacons to generate (default is 1).
    room_size : float, optional
        The size of the boundary within which positions are generated (default is 100).
    velocity : float, optional
        The speed at which agents move (default is 1.0).
    dt : float, optional
        The time step for the update (default is 0.1).
    influence_weight : float, optional
        The weight for influence_vector1 in determining new orientations (default is 0.7).
    sensing_radius : float, optional
        The sensing radius for the Vicsek model (default is 10.0).
    num_timesteps : int, optional
        The number of steps to simulate (default is 100).

    Returns
    -------
    tuple of np.ndarray
        The time series of positions and orientations of the agents.
    """

    if theta is not None:
        influence_weight = theta[0]
        sensing_radius = theta[1]
        velocity = theta[2]
        #internal_focus = theta[3]


    num_timesteps = int(time_horizon / dt)

    # Apply radial bound with sigmoid transformation for the sensing radius
    # (r_min, r_max) = (1., 5.)
    # sensing_radius = r_min + (r_max - r_min) * (1. / (1. + np.exp(-sensing_radius)))

    # Initialize arrays to store time series of positions and orientations
    for b in range(batch_size):
        positions = np.zeros((num_timesteps, num_agents, 2))
        rotations = np.zeros((num_timesteps, num_agents, ))
        neighbors = np.zeros((num_timesteps, num_agents, ))

            # Initialize positions and orientations
        initial_positions, initial_rotations = initialize_agents(num_agents, room_size=room_size)
        positions[0] = initial_positions
        rotations[0] = initial_rotations

        # Initialize beacons
        beacon_positions = initialize_beacons(num_beacons)

        # Simulation loop
        for t in range(1, num_timesteps):
            ps, rs, num_neighbors = combined_influences(
                agent_positions=positions[t-1],
                agent_rotations=rotations[t-1],
                beacon_positions=beacon_positions,
                velocity=velocity,
                sensing_radius=sensing_radius,
                dt=dt,
                influence_weight=influence_weight,
                internal_focus=internal_focus
            )

            # Store positions and orientations for each time step
            positions[t] = ps
            rotations[t] = rs
            neighbors[t] = num_neighbors

        neighbors[0] = neighbors[1]

        rotations = rotations[:,:,np.newaxis]
        neighbors = neighbors[:,:,np.newaxis]



    return np.concatenate((positions, rotations, neighbors), axis=-1)

In [23]:
class TogetherFlowSimulator:

    def __init__(self,
                 num_agents: int = 12,
                 num_beacons: int = 1,
                 room_size: tuple = (8, 10),
                 dt: float = 0.001,
                 internal_focus: float = 0.1,
                 time_horizon: float = 30.
                 ):
        self.num_agents = num_agents
        self.num_beacons = num_beacons
        self.room_size = room_size
        self.dt = dt
        self.internal_focus = internal_focus
        self.time_horizon = time_horizon


    def sample(self, batch_shape: int | tuple = (1,)) -> dict[str, np.ndarray]:

        batch_size = batch_shape[0]
        thetas = []
        samples = []

        for i in range(batch_size):
            theta = complete_pooling_prior()
            sim = simulator_fun(
                theta=theta,
                num_agents=self.num_agents,
                num_beacons=self.num_beacons,
                room_size=self.room_size,
                dt=self.dt,
                internal_focus=self.internal_focus,
                time_horizon=self.time_horizon
            )
            thetas.append(theta)
            samples.append(sim)

        thetas = np.array(thetas)
        samples = np.array(samples)

        return dict(
            w = thetas[:,0],
            r = thetas[:,1],
            v = thetas[:,2],
            positions = samples[:,:,:,0:2],
            rotations = samples[:,:,:,2],
            neighbors = samples[:,:,:,3]
        )

In [24]:
simulator = TogetherFlowSimulator(num_agents=49)

In [25]:
test_sims = simulator.sample(batch_shape=(10,))

In [26]:
print(test_sims['w'].shape)
print(test_sims['r'].shape)
print(test_sims['v'].shape)
print(test_sims['positions'].shape)
print(test_sims['rotations'].shape)
print(test_sims['neighbors'].shape)

(10,)
(10,)
(10,)
(10, 30000, 49, 2)
(10, 30000, 49)
(10, 30000, 49)


### Priors

### Adapter

In [27]:
adapter = (
    bf.adapters.Adapter()
    .convert_dtype("float64", "float32")
    .as_time_series(["positions", "rotations", "neighbors"])
    .expand_dims(['rotations', 'neighbors'], axis=-1)
    .expand_dims(['w', 'r', 'v'], axis=-1)
    .concatenate(['w', 'r', 'v'], into="inference_variables")
    .concatenate(["positions", "rotations", "neighbors"], into="summary_variables", axis=-1)
)

In [28]:
adapted_simulator = TogetherFlowSimulator(num_agents=49)
adapter_sim = adapter(adapted_simulator.sample(batch_shape=(2,)))

In [29]:
print(adapter_sim['summary_variables'].shape)
print(adapter_sim['inference_variables'].shape)

(2, 30000, 49, 4)
(2, 3)


### Neural Approximator

In [36]:
from bayesflow.networks import SummaryNetwork

In [37]:
class HierarchicalNetwork(SummaryNetwork):


    def __init__(self, networks, **kwargs):
        super().__init__(**kwargs)
        self.networks = networks

    def call(self, x, return_all=False, **kwargs):

        if return_all:
            outputs = []
            for net in self.networks:
                x = net(x, **kwargs)
                outputs.append(x)
            return outputs
        else:
            for net in self.networks:
                x = net(x, **kwargs)
            return x



In [38]:
# How do we go about the summary network, since HierarchicalNetwork has not been implemented yet?

# Need to ask Stefan about this.
summary_net = HierarchicalNetwork([
    keras.layers.TimeDistributed(tf.keras.layers.LSTM(units=256)),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=1))
])

In [39]:
inference_net = bf.networks.FlowMatching()

In [40]:
workflow = bf.workflows.BasicWorkflow(
    simulator=adapted_simulator,
    adapter=adapter,
    inference_network=inference_net,
    summary_network=summary_net,
)

In [ ]:
train_set = workflow.simulate((200,))
test_set = workflow.simulate((10,))

In [42]:
history = workflow.fit_offline(
    data=train_set,
    validation_set=test_set,
    batch_size=32,
    epochs=1
)

INFO:bayesflow:Fitting on dataset instance of OfflineDataset.
INFO:bayesflow:Building on a test batch.


KeyboardInterrupt: 

In [ ]:
plots = workflow.plot_default_diagnostics(

)